In [1]:
import ee
import geemap
import math
ee.Initialize()

## Imports

#### Datasets

In [2]:
ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
modis_ndvi = ee.ImageCollection("MODIS/006/MOD13Q1")
modis = ee.ImageCollection('MODIS/006/MCD43A4')
admin = ee.FeatureCollection("FAO/GAUL/2015/level0")
active_mines = ee.FeatureCollection("users/raymondeah/cod_mines_curated_all_opendata_p_ipis")

#### Visualization Parameters

In [3]:
rgb_vis_s2 = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
};

#### Geometry

In [4]:
roi1 = ee.Geometry.Polygon(
        [[[29.554129272985683, 3.1591674847348235],
          [29.554129272985683, 3.092319151883147],
          [29.625197083044277, 3.092319151883147],
          [29.625197083044277, 3.1591674847348235]]])

roi2 = ee.Geometry.Polygon(
        [[[30.246670960050185, 1.7911944738716732],
          [30.246670960050185, 1.7103797163160706],
          [30.356362579923232, 1.7103797163160706],
          [30.356362579923232, 1.7911944738716732]]])

all = ee.Geometry.Polygon(
        [[[22.92587933089792, 5.5911393992628495],
          [22.92587933089792, -13.776973382582892],
          [31.45126995589792, -13.776973382582892],
          [31.45126995589792, 5.5911393992628495]]])

big = ee.Geometry.Polygon(
         [[[29.324961334750263, 3.3062182501952346],
           [29.324961334750263, 2.9023783494297306],
           [29.879770905062763, 2.9023783494297306],
           [29.879770905062763, 3.3062182501952346]]], None, False)

big2 = ee.Geometry.Polygon(
        [[[23.93083329745621, 5.799063854338723],
          [23.93083329745621, -0.38729587002473354],
          [31.48942704745621, -0.38729587002473354],
          [31.48942704745621, 5.799063854338723]]])

big3 = ee.Geometry.Polygon(
        [[[26.156761924059015, -0.3200034949749593],
          [26.156761924059015, -2.4396289702937817],
          [29.276879111559015, -2.4396289702937817],
          [29.276879111559015, -0.3200034949749593]]])

small_scale = ee.Geometry.Polygon(
        [[[27.5104479108957, -0.8826212527375145],
          [27.5104479108957, -1.1599816977095712],
          [28.01307242261445, -1.1599816977095712],
          [28.01307242261445, -0.8826212527375145]]])

rishiArea = ee.Geometry.Polygon(
        [[[27.35, -7.5],
          [27.35, -7.6],
          [27.45, -7.6],
          [27.45, -7.5]]])

#### Training Data

In [5]:
bare = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.56666114888061, 3.1144301810591006]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.571982651566156, 3.139798234432259]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.586917191361078, 3.106373981962297]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.601680069779047, 3.1313994198763146]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.604426651810297, 3.122486318522175]),
            {
              "landcover": 1,
              "system:index": "4"
            })])

vegetation = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.59962013325561, 3.101745924882264]),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.617472916458734, 3.106031163614695]),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.614211350296625, 3.152482030389055]),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.55739143452514, 3.0947180957749514]),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.558078080032953, 3.101745924882264]),
            {
              "landcover": 0,
              "system:index": "4"
            })])

## Classification/Areal Change

In [6]:
true_color = modis.select([
  'Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band4',
  'Nadir_Reflectance_Band3'
])

composite2020 = true_color.filter(ee.Filter.date('2020-01-01', '2020-12-31')).median().clip(roi1)
composite2000 = true_color.filter(ee.Filter.date('2000-01-01', '2000-12-31')).median().clip(roi1)

true_color_vis = {
    'min': 0.0, 
    'max': 4000.0,
    'gamma': 1.4,
}

Map = geemap.Map()
Map.centerObject(roi1, 12)
Map.addLayer(composite2000, true_color_vis, '2000 Median Composite')
Map.addLayer(composite2020, true_color_vis, '2020 Median Composite')
Map.addLayer(bare, {'color': 'brown'}, 'Bare Points')
Map.addLayer(vegetation, {'color': 'green'}, 'Vegetation Points')
#Map

In [7]:
training = bare.merge(vegetation)

# Overlay the point on the image to get training data.
training = composite2020.sampleRegions(**{
  'collection': training,
  'properties': ['landcover'],
  'scale': 1
})

# Train a classifier.
classifier = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': composite2020.bandNames()
})

# Classify the image.
left_classified = composite2000.classify(classifier)
Map.addLayer(left_classified,
  {'min': 0, 'max': 1, 'palette': ['green', 'brown']}, '2000 Classified')

# Classify the image.
right_classified = composite2020.classify(classifier)
Map.addLayer(right_classified,
  {'min': 0, 'max': 1, 'palette': ['green', 'brown']}, '2020 Classified')

#Map

In [8]:
# Reclassify from 0-3 to 1-4
left_classes = left_classified.remap([0, 1], [1, 2])
right_classes = right_classified.remap([0, 1], [1, 2])

initial_vegetation = left_classes.eq(1)
bare = right_classes.eq(2)
vegetation_to_bare = initial_vegetation.And(bare)

Map.addLayer(vegetation_to_bare, {'min': 0, 'max': 1, 'palette': ['white', 'brown']}, 'vegetation to bare')
Map.centerObject(roi1, 12)
#Map

In [9]:
area_image = vegetation_to_bare.multiply(ee.Image.pixelArea())

area = area_image.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': roi1,
  'scale': 1000,
  'maxPixels': 1e10
})

area_vegetation_to_bare = ee.Number(area.get('remapped')).divide(1e6).getInfo()

area_initial_vegetation = left_classified.eq(0).multiply(ee.Image.pixelArea())

area2 = area_initial_vegetation.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': roi1,
  'scale': 1000,
  'maxPixels': 1e10
})

area_initial_vegetation = ee.Number(area2.get('classification')).divide(1e6).getInfo()

print('Vegetation area in 2000:', area_initial_vegetation, 'sq km')
print('Area of Vegetation -> Bare from 2000-2020:', area_vegetation_to_bare, 'sq km')
print('% Change:', area_vegetation_to_bare / area_initial_vegetation * 100, '%')

Vegetation area in 2000: 55.41621163088233 sq km
Area of Vegetation -> Bare from 2000-2020: 19.977357927941178 sq km
% Change: 36.049663699508834 %


In [10]:
def calculate_percentage_change(feature):
    g = feature.geometry()
    true_color = modis.select([
        'Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band4',
        'Nadir_Reflectance_Band3'
    ])

    composite2020 = true_color.filter(ee.Filter.date('2020-01-01', '2020-12-31')).median().clip(g)
    composite2000 = true_color.filter(ee.Filter.date('2000-01-01', '2000-12-31')).median().clip(g)
    
    left_classified = composite2000.classify(classifier)
    right_classified = composite2020.classify(classifier)
    
    # Reclassify from 0-3 to 1-4
    left_classes = left_classified.remap([0, 1], [1, 2])
    right_classes = right_classified.remap([0, 1], [1, 2])

    initial_vegetation = left_classes.eq(1)
    bare = right_classes.eq(2)
    vegetation_to_bare = initial_vegetation.And(bare)
    
    area_image = vegetation_to_bare.multiply(ee.Image.pixelArea())

    area = area_image.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': g,
      'scale': 1000,
      'maxPixels': 1e10
    })

    area_vegetation_to_bare = ee.Number(area.get('remapped')).divide(1e6)

    area_initial_vegetation = left_classified.eq(0).multiply(ee.Image.pixelArea())

    area2 = area_initial_vegetation.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': g,
      'scale': 1000,
      'maxPixels': 1e10
    })

    area_initial_vegetation = ee.Number(area2.get('classification')).divide(1e6)
    
    percent_loss = area_vegetation_to_bare.divide(area_initial_vegetation).multiply(100)
    
    return feature.set('percent loss',  percent_loss)

## Region Segmentation

In [11]:
"""
Segment the given geometry into squares of given size (in km)
:param geometry: rectangle form geometry object
:return: list including all squares
"""
def create_segments(geometry, size):
    segments = []
    r_earth, dy, dx, pi = ee.Number(6378), ee.Number(size), ee.Number(size), ee.Number(math.pi)
    
    coords = ee.List(geometry.coordinates().get(0)).slice(0, -1)
    
    top = ee.Number(ee.List(coords.get(2)).get(1))
    left = ee.Number(ee.List(coords.get(0)).get(0))
    
    width = int(ee.Geometry.Point(coords.get(0)).distance(ee.Geometry.Point(coords.get(1))).divide(1000 * size).getInfo())
    height = int(ee.Geometry.Point(coords.get(1)).distance(ee.Geometry.Point(coords.get(2))).divide(1000 * size).getInfo())

    for y in range(height + 1):
        left = ee.Number(ee.List(coords.get(0)).get(0))
        for x in range(width + 1):
            #
            first = top
            second = dx.divide(r_earth)
            third = ee.Number(180).divide(pi)
            con = pi.divide(ee.Number(180))
            fourth = left.multiply(con).multiply(con).cos()
            
            new_lon = first.subtract(second.multiply(third).divide(fourth))
            #new_lon = top - (dx / r_earth) * (180 / pi) / math.cos(math.radians(left * pi/180))
            #new_lat = left  + (dy / r_earth) * (180 / pi)
            new_lat = left.add((dy.divide(r_earth)).multiply((ee.Number(180).divide(pi))))
            
            square = ee.Geometry.Polygon(
                [[[left, new_lon],
                  [new_lat, new_lon],
                  [new_lat, top],
                  [left, top]]], None, False)
            
            segments.append(square)
            
            left = new_lat
        top = new_lon
        
    return segments

## Reference Median Composite

In [12]:
def create_median_composite(feature):
    filtered = s2 \
        .filter(ee.Filter.bounds(feature.geometry())) \
        .filter(ee.Filter.date('2020-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*')
    
    composite = filtered.median().clip(feature.geometry())
    return composite

## Workflow

In [13]:
def filter_by_vegetation_loss(squares, threshold):
    with_percent_change = squares.map(calculate_percentage_change)
    passed = with_percent_change.filter(ee.Filter.gt('percent loss', threshold))
    return passed

In [28]:
Map = geemap.Map()
Map.centerObject(roi1, 12)

drc = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))

segments = ee.FeatureCollection(create_segments(roi1, 0.5)).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
composites = ee.ImageCollection(segments.map(create_median_composite))

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

Map

Map(center=[3.1257435633122834, 29.589663178011996], controls=(WidgetControl(options=['position', 'transparent…

In [25]:
Map = geemap.Map()
Map.centerObject(roi2, 12)

drc = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))

segments = ee.FeatureCollection(create_segments(roi2, 0.5)).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
# read = passed_vegetation_loss.getInfo()['features']
# for item in read:
#     print(item['properties'])

composites = ee.ImageCollection(segments.map(create_median_composite))

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

Map

Map(center=[1.7507876063752705, 30.30151676998732], controls=(WidgetControl(options=['position', 'transparent_…

In [26]:
Map = geemap.Map()
Map.centerObject(rishiArea, 12)

drc = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))

segments = ee.FeatureCollection(create_segments(rishiArea, 0.5)).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
composites = ee.ImageCollection(segments.map(create_median_composite))

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

Map

Map(center=[-7.55000091393896, 27.399999999997924], controls=(WidgetControl(options=['position', 'transparent_…

In [27]:
emily_area = ee.Geometry.Polygon(
        [[[25.599767416235804, -10.585427828449394],
          [25.599767416235804, -10.82359555219894],
          [25.977297323462366, -10.82359555219894],
          [25.977297323462366, -10.585427828449394]]])

Map = geemap.Map()
Map.centerObject(emily_area, 11)

drc = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))

segments = ee.FeatureCollection(create_segments(emily_area, 0.5)).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
composites = ee.ImageCollection(segments.map(create_median_composite))

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

Map

Map(center=[-10.704552846449875, 25.788532369849722], controls=(WidgetControl(options=['position', 'transparen…

In [18]:
# Map = geemap.Map()
# Map.centerObject(all, 5)

# drc = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))

# segments = create_segments(all, 10)
# mid = len(segments) // 2
# first_half = ee.FeatureCollection(segments[:mid]).filter(ee.Filter.bounds(drc))
# second_half = ee.FeatureCollection(segments[mid:]).filter(ee.Filter.bounds(drc))

# passed_vegetation_loss1 = filter_by_vegetation_loss(first_half, 0)
# passed_vegetation_loss2 = filter_by_vegetation_loss(second_half, 0)

# composites1 = ee.ImageCollection(first_half.map(create_median_composite))
# composites2 = ee.ImageCollection(second_half.map(create_median_composite))

# Map.addLayer(all, {'color': 'blue'}, 'bounding box')
# Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')


# Map.addLayer(composites1, rgb_vis_s2, 'Median Composites 2021 1')
# Map.addLayer(composites2, rgb_vis_s2, 'Median Composites 2021 2')

# Map.addLayer(passed_vegetation_loss1, {'color': 'red'}, 'Passed 0% Vegetation Loss 1')
# Map.addLayer(passed_vegetation_loss2, {'color': 'red'}, 'Passed 0% Vegetation Loss 2')

# Map

## Exporting Results

In [19]:
# task = ee.batch.Export.table.toDrive(collection=passed2,
#                                      description='first pass 2',
#                                      fileFormat='SHP')
# task.start()

In [20]:
# task.status()

## Indices

In [21]:
def ndwi(f):
    return f.normalizedDifference(['B3', 'B11'])#.rename('ndwi')

def nir_g(f):
    return f.normalizedDifference(['B8', 'B3'])

def nir_swir2(f):
    return f.normalizedDifference(['B8', 'B12'])

composites = ee.ImageCollection(ee.FeatureCollection(create_segments(roi1, 1)).map(create_median_composite))

ndwi = composites.map(ndwi)

# nir_g1 = composites1.map(nir_g)
# nir_g2 = composites2.map(nir_g)

# nir_swir1 = composites1.map(nir_swir2)
# nir_swir2 = composites2.map(nir_swir2)

ndwi_vis = {'min': 0, 'max': 1, 'palette': ['white', 'blue']}
nir_g_vis = {'min': 0, 'max': 1, 'palette': ['white', 'red']}

Map = geemap.Map()
Map.centerObject(roi1, 12)
Map.addLayer(composites, rgb_vis_s2, 'Median Composite')
Map.addLayer(ndwi, ndwi_vis, 'NDWI')

# Map.addLayer(nir_g1, nir_g_vis, 'NIR/G 1')
# Map.addLayer(nir_g2, nir_g_vis, 'NIR/G 2')
# Map.addLayer(nir_swir1, nir_g_vis, 'NIR/SWIR2 1')
# Map.addLayer(nir_swir2, nir_g_vis, 'NIR/SWIR2 2')
Map

Map(center=[3.1257435633122834, 29.589663178011996], controls=(WidgetControl(options=['position', 'transparent…